In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [100]:
world_cup = pd.read_csv('World Cup 2019 Dataset.csv',encoding="windows-1252",sep = ',')
results = pd.read_csv('T201.csv', encoding="windows-1252",sep = ',')
ranking = pd.read_csv('icc_rankings.csv',encoding="windows-1252") 
fixtures = pd.read_csv('fixtures.csv',encoding="windows-1252")

In [101]:
ranking.head()

,Position,Team,Points
0,1,England,125
1,2,India,121
2,3,South Africa,115
3,4,New Zealand,113
4,5,Australia,109


In [102]:
world_cup.head()

,Team,Group,Previous \r\nappearances,Previous \r\ntitles,Previous\r\n finals,Previous\r\n semifinals,Current \r rank
0,England,A,11,0,3,5,1
1,South Africa,A,6,0,0,4,3
2,West Indies,A,11,2,3,4,8
3,Pakistan,A,11,1,2,6,6
4,New Zealand,A,11,0,1,7,4


In [103]:
results.head()

,Team1,Team2,Ground,Toss,Bat,Winner
0,Pakistan,Bangladesh,Karachi,win,first,Pakistan
1,England,New Zealand,Manchester,win,2nd,England
2,West Indies,Australia,Bridgetown,win,2nd,West Indies
3,Netherlands,Kenya,Belfast,loss,first,Netherlands
4,Ireland,Scotland,Belfast,loss,2nd,Ireland


In [104]:
fixtures.head()

,Round Number,Date,Location,Team_1,Team_2,Group,Result
0,1,30/05/2019,"Kennington Oval, London",England,South Africa,Group A,NaN
1,1,31/05/2019,"Trent Bridge, Nottingham",West Indies,Pakistan,Group A,NaN
2,1,1/6/2019,"Sophia Gardens, Cardiff",New Zealand,Sri Lanka,Group A,NaN
3,1,1/6/2019,"County Ground, Bristol",Afghanistan,Australia,Group A,NaN
4,1,2/6/2019,"Kennington Oval, London",South Africa,Bangladesh,Group A,NaN


In [105]:
df = results[(results['Team1'] == 'India') | (results['Team2'] == 'India')]
india = df.iloc[:]
india.head()

,Team1,Team2,Ground,Toss,Bat,Winner
24,India,Sri Lanka,Colombo (RPS),loss,2nd,India
35,India,Bangladesh,Nottingham,win,first,India
43,India,Ireland,Nottingham,win,2nd,India
68,India,Sri Lanka,Mohali,loss,2nd,India
93,India,Afghanistan,Gros Islet,win,2nd,India


In [107]:
year = []
for row in india['Team1']:
    year.append(int(row[7:]))
india ['match_year']= year
india_2010 = india[india.match_year >= 10]
india_2010.count()

ValueError: invalid literal for int() with base 10: ''

In [109]:
worldcup_teams = ['England', ' South Africa', '', 'West Indies', 
            'Pakistan', 'New Zealand', 'Sri Lanka', 'Afghanistan', 
            'Australia', 'Bangladesh', 'India','Papua New Guinea','Ireland','Oman','Namibia','Scotland','Netherlands']
df_teams_1 = results[results['Team1'].isin(worldcup_teams)]
df_teams_2 = results[results['Team2'].isin(worldcup_teams)]
df_teams = pd.concat((df_teams_1, df_teams_2))
df_teams.drop_duplicates()
df_teams.count()

Team1     643
Team2     643
Ground    643
Toss      643
Bat       643
Winner    643
dtype: int64

In [110]:
#df_teams_2010 = df_teams.drop(['Scorecard','Margin','Ground','Match Date','Host_Country','Venue_Team1','Venue_Team2','Innings_Team1','Innings_Team2',''], axis=1)
#df_teams_2010.head()  # use for result dataset
#df_teams_2010 = df_teams.drop(['Result','Margin','Ground','Start Date','Host_Country','Venue_Team 1','Venue_Team 2','Inning_Team1','Inning_Team2','Team1_Toss','Team2_Toss','pitch','Winner_Inning','pitchn','Toss','Bat'], axis=1)
#df_teams_2010.head()  # use for 2000 to 2019 dataset
df_teams_2010 = df_teams.drop(['Ground','Toss','Bat'], axis=1)
df_teams_2010.head() 


,Team1,Team2,Winner
0,Pakistan,Bangladesh,Pakistan
1,England,New Zealand,England
2,West Indies,Australia,West Indies
3,Netherlands,Kenya,Netherlands
4,Ireland,Scotland,Ireland


In [111]:
df_teams_2010 = df_teams_2010.reset_index(drop=True)
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team1,'winning_team']=1
df_teams_2010.loc[df_teams_2010.Winner == df_teams_2010.Team2, 'winning_team']=2
df_teams_2010 = df_teams_2010.drop(['winning_team'], axis=1)

df_teams_2010.head()

,Team1,Team2,Winner
0,Pakistan,Bangladesh,Pakistan
1,England,New Zealand,England
2,West Indies,Australia,West Indies
3,Netherlands,Kenya,Netherlands
4,Ireland,Scotland,Ireland


In [112]:
final = pd.get_dummies(df_teams_2010, prefix=['Team_1', 'Team_2'], columns=['Team1', 'Team2'])

X = final.drop(['Winner'], axis=1)
y = final["Winner"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [113]:
rf = RandomForestClassifier(n_estimators=100, max_depth=20,
                              random_state=0)
rf.fit(X_train, y_train) 


score = rf.score(X_train, y_train)
score2 = rf.score(X_test, y_test)


print("Training set accuracy: ", '%.3f'%(score))
print("Test set accuracy: ", '%.3f'%(score2))

Training set accuracy:  1.000
Test set accuracy:  1.000


In [114]:
#ranking = pd.read_csv('icc_rankings1.csv',encoding="windows-1252") 
#fixtures = pd.read_csv('fixtures2.csv',encoding="windows-1252")
pred_set = []

In [115]:
fixtures.insert(1, 'first_position', fixtures['Team_1'].map(ranking.set_index('Team')['Position']))
fixtures.insert(2, 'second_position', fixtures['Team_2'].map(ranking.set_index('Team')['Position']))

fixtures = fixtures.iloc[:45, :]
fixtures.tail()

,Round Number,first_position,second_position,Date,Location,Team_1,Team_2,Group,Result
40,1,1.0,4.0,3/7/2019,"Riverside Ground, Chester-le-Street",England,New Zealand,Group A,NaN
41,1,10.0,8.0,4/7/2019,"Headingley, Leeds",Afghanistan,West Indies,Group A,NaN
42,1,6.0,7.0,5/7/2019,"Lord's, London",Pakistan,Bangladesh,Group A,NaN
43,1,9.0,2.0,6/7/2019,"Headingley, Leeds",Sri Lanka,India,Group A,NaN
44,1,5.0,3.0,6/7/2019,"Emirates Old Trafford, Manchester",Australia,South Africa,Group A,NaN


In [116]:
for index, row in fixtures.iterrows():
    if row['first_position'] < row['second_position']:
        pred_set.append({'Team_1': row['Team_1'], 'Team_2': row['Team_2'], 'winning_team': None})
    else:
        pred_set.append({'Team_1': row['Team_2'], 'Team_2': row['Team_1'], 'winning_team': None})
        
pred_set = pd.DataFrame(pred_set)
backup_pred_set = pred_set
pred_set.head()

,Team_1,Team_2,winning_team
0,England,South Africa,None
1,Pakistan,West Indies,None
2,New Zealand,Sri Lanka,None
3,Australia,Afghanistan,None
4,South Africa,Bangladesh,None


In [117]:
pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

missing_cols = set(final.columns) - set(pred_set.columns)
for c in missing_cols:
    pred_set[c] = 0
pred_set = pred_set[final.columns]


pred_set = pred_set.drop(['Winner'], axis=1)
pred_set.head()

,Team_1_Afghanistan,Team_1_Australia,Team_1_Bangladesh,Team_1_England,Team_1_India,Team_1_Ireland,Team_1_Namibia,Team_1_Netherlands,Team_1_New Zealand,Team_1_Oman,...,Team_2_ Pakistan,Team_2_ Scotland,Team_2_ Singapore,Team_2_ South Africa,Team_2_ Sri Lanka,Team_2_ U.A.E.,Team_2_ Uganda,Team_2_ West Indies,Team_2_ World-XI,Team_2_ Zimbabwe
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [118]:
predictions = rf.predict(pred_set)
for i in range(fixtures.shape[0]):
    print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
    if predictions[i] == 1:
        print("Winner: " + backup_pred_set.iloc[i, 1])
    
    else:
        print("Winner: " + backup_pred_set.iloc[i, 0])
    print("")

South Africa and England
Winner: England

West Indies and Pakistan
Winner: Pakistan

Sri Lanka and New Zealand
Winner: New Zealand

Afghanistan and Australia
Winner: Australia

Bangladesh and South Africa
Winner: South Africa

Pakistan and England
Winner: England

Afghanistan and Sri Lanka
Winner: Sri Lanka

South Africa and India
Winner: India

Bangladesh and New Zealand
Winner: New Zealand

West Indies and Australia
Winner: Australia

Sri Lanka and Pakistan
Winner: Pakistan

Bangladesh and England
Winner: England

Afghanistan and New Zealand
Winner: New Zealand

Australia and India
Winner: India

West Indies and South Africa
Winner: South Africa

Sri Lanka and Bangladesh
Winner: Bangladesh

Pakistan and Australia
Winner: Australia

New Zealand and India
Winner: India

West Indies and England
Winner: England

Sri Lanka and Australia
Winner: Australia

Afghanistan and South Africa
Winner: South Africa

Pakistan and India
Winner: India

West Indies and Bangladesh
Winner: Bangladesh

Afg

In [119]:
# List of tuples before 
semi = [('New Zealand', 'India'),
            ('England', 'South Africa')]

In [120]:
def clean_and_predict(matches, ranking, final, logreg):

    # Initialization of auxiliary list for data cleaning
    positions = []

    # Loop to retrieve each team's position according to ICC ranking
    for match in matches:
        positions.append(ranking.loc[ranking['Team'] == match[0],'Position'].iloc[0])
        positions.append(ranking.loc[ranking['Team'] == match[1],'Position'].iloc[0])
    
    # Creating the DataFrame for prediction
    pred_set = []

    # Initializing iterators for while loop
    i = 0
    j = 0

    # 'i' will be the iterator for the 'positions' list, and 'j' for the list of matches (list of tuples)
    while i < len(positions):
        dict1 = {}

        # If position of first team is better then this team will be the 'Team_1' team, and vice-versa
        if positions[i] < positions[i + 1]:
            dict1.update({'Team_1': matches[j][0], 'Team_2': matches[j][1]})
        else:
            dict1.update({'Team_1': matches[j][1], 'Team_2': matches[j][0]})

        # Append updated dictionary to the list, that will later be converted into a DataFrame
        pred_set.append(dict1)
        i += 2
        j += 1
        
        # Convert list into DataFrame
    pred_set = pd.DataFrame(pred_set)
    backup_pred_set = pred_set

    # Get dummy variables and drop winning_team column
    pred_set = pd.get_dummies(pred_set, prefix=['Team_1', 'Team_2'], columns=['Team_1', 'Team_2'])

    # Add missing columns compared to the model's training dataset
    missing_cols2 = set(final.columns) - set(pred_set.columns)
    for c in missing_cols2:
        pred_set[c] = 0
    pred_set = pred_set[final.columns]

    pred_set = pred_set.drop(['Winner'], axis=1)

    # Predict!
    predictions = logreg.predict(pred_set)
    for i in range(len(pred_set)):
        print(backup_pred_set.iloc[i, 1] + " and " + backup_pred_set.iloc[i, 0])
        if predictions[i] == 1:
            print("Winner: " + backup_pred_set.iloc[i, 1])
        else:
            print("Winner: " + backup_pred_set.iloc[i, 0])
        print("")

In [121]:
clean_and_predict(semi, ranking, final, rf)

New Zealand and India
Winner: India

South Africa and England
Winner: England

